# Add punctuation to text:
### given a dataframe with the text in one column:

In [ ]:
!pip install deepmultilingualpunctuation

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import pandas as pd

df = pd.read_csv('noise.csv')


In [ ]:
import re
from deepmultilingualpunctuation import PunctuationModel
from tqdm import tqdm

new = []
for t in tqdm(df['text']):
  model = PunctuationModel()
  result = model.restore_punctuation(t)
  new.append(new)


df['text_with_punctuation'] = new


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


# create the embeddings

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import re
model_embedding = SentenceTransformer('sentence-transformers/sentence-t5-base')

def split_text(text):
  split_text = re.split(r'[,.?!:;]', text)
  split_text = [part.strip() for part in split_text if part.strip()]
  return split_text


def generate_embeddings(text_list, model):
  return model.encode(text_list)


In [ ]:
df['text'] = df['text'].apply(split_text)
df['embeddings'] = df['text'].apply(generate_embeddings)


# Pytorch model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def pad_and_convert_to_tensor(sequences, max_len=None, pad_value=0.0):

    if max_len is None:
        max_len = max(len(seq) for seq in sequences)

    padded_sequences = []
    for seq in sequences:
        padded_seq = np.pad(seq, ((0, max_len - len(seq)), (0, 0)), mode='constant', constant_values=pad_value)
        padded_sequences.append(padded_seq)

    padded_sequences = np.array(padded_sequences)
    return torch.tensor(padded_sequences, dtype=torch.float32)

class Masking(nn.Module):
    def __init__(self, mask_value=0.0):
        super(Masking, self).__init__()
        self.mask_value = mask_value

    def forward(self, x):
        mask = (x != self.mask_value).float()
        return x * mask, mask


class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        # x: (batch_size, time_steps, input_size)
        batch_size, time_steps, input_size = x.size()

        # Reshape input tensor to (batch_size * time_steps, input_size)
        x = x.contiguous().view(batch_size * time_steps, input_size)

        # Apply the module
        x = self.module(x)

        # Reshape the output to (batch_size, time_steps, output_size)
        output_size = x.size(-1)
        x = x.contiguous().view(batch_size, time_steps, output_size)

        return x

class BiRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_heads=4, num_layers=1, mask_value=0.0):
        super(BiRNNModel, self).__init__()

        self.masking = Masking(mask_value=mask_value)
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size*2, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.time_distributed = TimeDistributed(nn.Linear(hidden_size * 2, output_size))

    def forward(self, x):
        x, mask = self.masking(x)
        x, _ = self.rnn(x)

        # Permute to match the transformer's expected input shape (sequence_length, batch_size, hidden_size*2)
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)

        # Permute back to original shape (batch_size, sequence_length, hidden_size*2)
        x = x.permute(1, 0, 2)

        x = self.time_distributed(x)
        return x, mask


In [ ]:
padded_tensor = pad_and_convert_to_tensor(sequences, pad_value=0.0)


# Parameters
input_size = 10  # Number of input features
hidden_size = 20 # Number of features in the hidden state
output_size = 2  # Number of output features
mask_value = 0.0

# Initialize the model
model = BiRNNModel(input_size, hidden_size, output_size, mask_value)

# Dummy input (batch_size, time_steps, input_size)
dummy_input = torch.tensor([[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                             [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]], dtype=torch.float32)

# Forward pass
output, mask = model(dummy_input)

print("Output:", output)
print("Mask:", mask)